In [1]:
%%writefile oneapi_matrix_multiply.cpp
//==============================================================
// Matrix Multiplication using OneAPI: SYCL
//==============================================================
// Copyright © 2024 Your Name
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <CL/sycl.hpp>          // Incluye la biblioteca SYCL
#include <iostream>             // Incluye la biblioteca para entrada/salida estándar
#include <vector>               // Incluye la biblioteca para trabajar con vectores
#include <fstream>              // Incluye la biblioteca para trabajar con archivos

using namespace cl::sycl;       // Utiliza el espacio de nombres SYCL

int main(int argc, char* argv[]) {
    // Verifica si el número de argumentos es correcto
    if (argc != 2) {
        std::cerr << "Usage: " << argv[0] << " <output_file>" << std::endl;
        return 1;
    }

    const size_t N = 4;    // Tamaño de las matrices (N x N)

    // Inicializar matrices de entrada
    std::vector<float> matrix_a(N * N, 2.0f);   // Matriz A llena de 2.0
    std::vector<float> matrix_b(N * N, 3.0f);   // Matriz B llena de 3.0
    std::vector<float> matrix_c(N * N, 0.0f);   // Matriz C inicializada en 0.0

    try {
        // Crear un objeto queue para ejecutar tareas SYCL en el dispositivo predeterminado
        queue my_queue;

        // Crear buffers para las matrices en el dispositivo
        buffer<float, 2> buffer_a(matrix_a.data(), range<2>(N, N));
        buffer<float, 2> buffer_b(matrix_b.data(), range<2>(N, N));
        buffer<float, 2> buffer_c(matrix_c.data(), range<2>(N, N));

        // Ejecutar el kernel de multiplicación de matrices
        my_queue.submit([&](handler& h) {
            auto a = buffer_a.get_access<access::mode::read>(h);
            auto b = buffer_b.get_access<access::mode::read>(h);
            auto c = buffer_c.get_access<access::mode::write>(h);

            h.parallel_for<class matrix_multiply>(range<2>(N, N), [=](id<2> index) {
                float sum = 0.0f;
                for (size_t k = 0; k < N; ++k) {
                    sum += a[index[0]][k] * b[k][index[1]];  // Calcula la suma de la multiplicación de los elementos correspondientes
                }
                c[index] = sum;   // Asigna el resultado de la multiplicación a la matriz C
            });
        });

        // Esperar a que todas las tareas en la cola se completen
        my_queue.wait();

        // Imprimir la matriz resultante en la consola y escribirla en un archivo
        std::ofstream output_file(argv[1]);   // Abre un archivo para escribir la salida
        for (size_t i = 0; i < N; ++i) {
            for (size_t j = 0; j < N; ++j) {
                std::cout << matrix_c[i * N + j] << " ";   // Imprime cada elemento en la consola
                output_file << matrix_c[i * N + j] << " "; // Escribe cada elemento en el archivo
            }
            std::cout << "\n";        // Salto de línea en la consola
            output_file << "\n";      // Salto de línea en el archivo
        }
        output_file.close();   // Cierra el archivo de salida
    } catch (const cl::sycl::exception& e) {
        std::cerr << "SYCL Exception: " << e.what() << std::endl;   // Captura y muestra excepciones de SYCL
        return 1;
    }

    return 0;   // Retorna 0 si la ejecución es exitosa
}





Overwriting oneapi_matrix_multiply.cpp


In [2]:
!icpx -o matrix_multiply oneapi_matrix_multiply.cpp -fsycl


In [3]:
!./matrix_multiply output.txt


24 24 24 24 
24 24 24 24 
24 24 24 24 
24 24 24 24 


In [22]:
!chmod +x q


In [ ]:
!./q oneapi_matrix_multiply.cpp


In [26]:
!chmod +x oneapi_matrix_multiply.cpp

In [ ]:
!./q oneapi_matrix_multiply.cpp

In [ ]:
! chmod 755 q; chmod 755 run_oneapi_matrix_multiply.sh;if [ -x "$(command -v qsub)" ]; then ./q run_oneapi_matrix_multiply.sh; else ./run_oneapi_matrix_multiply.sh; fi